<a href="https://colab.research.google.com/github/Mohan0332/Capstone/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Implementation of the exisiting model

In [ ]:
import numpy as np
from PIL import Image
import os

In [ ]:
# function to convert a image to grayscale 
def rgb_to_gray(rgb_img):
    # Convert the image to a NumPy array
    rgb_array = np.array(rgb_img)

    # Create an empty grayscale image array with the same dimensions as the RGB image
    gray_array = np.empty(rgb_array.shape[:2], dtype=np.uint8)

    # Calculate the luminance value for each pixel in the RGB image and assign it to the grayscale image array
    for i in range(rgb_array.shape[0]):
        for j in range(rgb_array.shape[1]):
            r, g, b = rgb_array[i,j]
            luminance = int(0.2989 * r + 0.5870 * g + 0.1140 * b)
            gray_array[i,j] = luminance
    
    gray_img = Image.fromarray(gray_array)
    return gray_img

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = []
labels = []
classes = 2
cur_path = '/content/drive/MyDrive/Capstone dataset/Images-out_RGB'

#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '/'+ a)
            image = image.resize((100,100))
            img = rgb_to_gray(image)
            img = np.array(img)
            img = img.reshape((100, 100, 1))
            #image = np.array(image)
            #sim = Image.fromarray(image)
            data.append(img)
            labels.append(i)
        except:
            print("Error loading image")

In [ ]:
data = np.array(data)
labels = np.array(labels)

In [ ]:
print(data.shape, labels.shape)

(490, 100, 100, 1) (490,)


In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [ ]:
#Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.22, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(382, 100, 100, 1) (108, 100, 100, 1) (382,) (108,)


In [ ]:
#Converting the labels into one hot encoding
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [ ]:
#Building the model
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(100,100,1)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

In [ ]:
#Compilation of the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 98, 98, 16)        160       
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 49, 49, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 23, 23, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_30 (Conv2D)          (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 10, 10, 64)      

In [ ]:
epochs = 15
history = model.fit(X_train, y_train, batch_size=64, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/15
6/6 [==============================] - 8s 994ms/step - loss: 0.6130 - accuracy: 0.6885 - val_loss: 0.5641 - val_accuracy: 0.7222
Epoch 2/15
6/6 [==============================] - 3s 568ms/step - loss: 0.4944 - accuracy: 0.7775 - val_loss: 0.5772 - val_accuracy: 0.7037
Epoch 3/15
6/6 [==============================] - 3s 552ms/step - loss: 0.5099 - accuracy: 0.7723 - val_loss: 0.5208 - val_accuracy: 0.7222
Epoch 4/15
6/6 [==============================] - 3s 534ms/step - loss: 0.4475 - accuracy: 0.8037 - val_loss: 0.5917 - val_accuracy: 0.6667
Epoch 5/15
6/6 [==============================] - 5s 944ms/step - loss: 0.4324 - accuracy: 0.8141 - val_loss: 0.5245 - val_accuracy: 0.7315
Epoch 6/15
6/6 [==============================] - 3s 566ms/step - loss: 0.3792 - accuracy: 0.8508 - val_loss: 0.5750 - val_accuracy: 0.7037
Epoch 7/15
6/6 [==============================] - 3s 561ms/step - loss: 0.3229 - accuracy: 0.8743 - val_loss: 0.5259 - val_accuracy: 0.7222
Epoch 8/15
6/6 [====

In [ ]:
test_data = []
test_labels = []
classes = 2
cur_path = '/content/drive/MyDrive/Capstone dataset/Images-out_RGB'

#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'test',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '/'+ a)
            image = image.resize((100,100))
            img = rgb_to_gray(image)
            img = np.array(img)
            img = img.reshape((100, 100, 1))
            #sim = Image.fromarray(image)
            test_data.append(img)
            test_labels.append(i)
        except:
            print("Error loading image")
     


In [ ]:
x_test=np.array(test_data)

In [ ]:
x_test.shape

(57, 100, 100, 1)

In [ ]:
y_pred_prob = model.predict(x_test)
y_pred = np.argmax(y_pred_prob, axis=1)

2/2 [==============================] - 0s 58ms/step


In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_labels,y_pred ))

0.7192982456140351
